# Perspectief 1: Verschillen in milieubeleid verklaren verschillen in landbouwuitstoot

De uitstoot van de landbouwsector verschilt sterk tussen landen. In dit perspectief onderzoeken we hoe beleidskeuzes zoals milieuregels en landbouwsubsidies bijdragen aan die verschillen. We gebruiken internationale data om te analyseren welke landen efficiënt produceren en welke landen vervuilende landbouw blijven stimuleren.


### Argument 1: Landen met strenger milieubeleid stoten gemiddeld minder CO₂ uit per ton landbouwproductie.

In veel gesprekken over landbouw en klimaat gaat de aandacht uit naar de totale uitstoot per land. Maar het zegt veel meer als je kijkt naar hoe efficiënt die uitstoot is in verhouding tot wat er wordt geproduceerd. Door CO₂-uitstoot te koppelen aan de hoeveelheid landbouwproducten krijgen we een eerlijker beeld van hoe vervuilend of juist duurzaam een landbouwsysteem echt is. Zo kunnen landen op een eerlijke manier met elkaar worden vergeleken, ongeacht hun grootte of landbouwproductie, en wordt duidelijker welke landen daadwerkelijk inzetten op een duurzamere landbouw.


In [9]:
# Cell 1
import os
# make sure we're in /Users/carmenvanderlans/Information-Vis
os.chdir(os.path.expanduser("~/Information-Vis"))
print("cwd =", os.getcwd())

cwd = /Users/carmenvanderlans/Information-Vis


In [10]:
# — Cell 2: load, process, and draw the figure —

import pandas as pd
import plotly.graph_objects as go

# 1) Load raw data from data1/
emissies  = pd.read_csv("data1/FAOSTAT_landbouw_emissies_2022.csv")
productie = pd.read_csv("data1/FAOSTAT_landbouw_productie_2022.csv")
epi       = pd.read_csv("data1/epi2022results05302022.csv")

# 2) Aggregate to per‐country
em_land = emissies.groupby("Area")["Value"].sum().reset_index()
em_land.columns = ["Country", "Emissions_kt"]
pr_land = productie.groupby("Area")["Value"].sum().reset_index()
pr_land.columns = ["Country", "Production_ton"]
pr_land["Production_kt"] = pr_land["Production_ton"] / 1_000

# 3) Merge and compute per‐ton emissions
df = pd.merge(em_land, pr_land, on="Country")
df["Emissions_per_ton"] = df["Emissions_kt"] / df["Production_kt"]

# 4) Attach EPI score
epi = epi.rename(columns={"country":"Country", "EPI.new":"EPI_Score"})
df = pd.merge(df, epi[["Country","EPI_Score"]], on="Country", how="left")

# 5) Filter out very small producers
df = df[(df["Production_kt"] > 5_000) & (df["Emissions_kt"] > 50)]

# 6) Build the two layers
p10, p90 = df["Emissions_per_ton"].quantile([0.1, 0.9])

choropleth = go.Choropleth(
    locations=df["Country"], locationmode="country names",
    z=df["EPI_Score"], colorscale="YlGn", zmin=0, zmax=100,
    showscale=True, colorbar=dict(title="EPI Score", x=1.02, len=0.6)
)
scatter = go.Scattergeo(
    locations=df["Country"], locationmode="country names",
    text=df["Country"],
    marker=dict(
        size=6,
        color=df["Emissions_per_ton"],
        colorscale=["#ffffcc","#ffcc66","#ff9933","#cc3300","#660000"],
        cmin=p10, cmax=p90,
        colorbar=dict(
            title="CO₂ per ton",
            orientation="h",
            x=0.5, xanchor="center",
            y=-0.18, yanchor="bottom",
            len=1.0, thickness=15,
            tickvals=[round(p10,3), round((p10+p90)/2,3), round(p90,3)]
        )
    ),
    name="Uitstoot per ton"
)

# 7) Layout and annotations
fig = go.Figure([choropleth, scatter])
fig.update_layout(
    title="CO₂-uitstoot per ton landbouwproductie en milieubeleid (EPI-score) per land (2022)",
    title_font=dict(size=16),
    geo=dict(
        showframe=False, showcoastlines=True,
        projection_type="equirectangular", projection_scale=1.3,
        center=dict(lat=10, lon=10)
    ),
    margin=dict(t=60, b=220, l=60, r=60),
    height=710, width=1100,
    annotations=[
      dict(
        text="CO₂-uitstoot per ton landbouwproductie",
        showarrow=False, xref="paper", yref="paper",
        x=0.5, y=-0.15, xanchor="center", yanchor="bottom",
        font=dict(size=13)
      ),
      dict(
        text=(
          "Deze visualisatie toont de CO₂-uitstoot per ton landbouwproductie en relateert dit aan het milieubeleid van landen<br>"
          "zoals gemeten via de EPI-score. Alleen landen met voldoende landbouwproductie zijn meegenomen<br>"
          "(meer dan 5.000 kiloton productie en meer dan 50 kiloton uitstoot),<br>"
          "om vertekende resultaten door kleine landbouwlanden zoals Qatar te voorkomen."
        ),
        showarrow=False, xref="paper", yref="paper",
        x=0.5, y=-0.36, xanchor="center", yanchor="bottom",
        align="center", font=dict(size=12)
      )
    ]
)

# 8) Render
fig.show()


De visualisatie laat duidelijk zien dat landen met een hoge EPI-score, wat wijst op sterker milieubeleid, gemiddeld minder CO₂ uitstoten per ton landbouwproductie. Dit is goed te zien bij landen als Zwitserland, Duitsland en Nieuw-Zeeland. Deze landen zijn donkergroen gekleurd, wat duidt op een hoog milieubeleidsscore, en hebben lichtere stippen, wat betekent dat hun uitstoot per ton landbouwproductie relatief laag is.

Landen met lage EPI-scores, zoals India, Pakistan, Nigeria en Kenya, vallen juist op door hun lichtere achtergrondkleur in combinatie met donkergekleurde stippen. Dit betekent dat deze landen minder streng milieubeleid voeren en tegelijkertijd relatief veel uitstoten per ton landbouwproductie. Dit patroon ondersteunt sterk het idee dat strenger milieubeleid samenhangt met duurzamere landbouw.

Een opvallende uitzondering vormen sommige Scandinavische landen, zoals Zweden en Finland. Zij scoren hoog op milieubeleid en zijn dus donkergroen op de kaart, maar hebben toch relatief donkere stippen. Dat betekent dat hun uitstoot per ton landbouwproductie hoger ligt dan je op basis van hun EPI-score zou verwachten. Toch zijn dit uitzonderingen op een veel duidelijker en bredere trend: landen met beter milieubeleid produceren landbouw gemiddeld op een schonere en efficiëntere manier.

### Argument 2: Landen die duurzaamheid belonen via subsidies, stoten relatief minder CO₂ uit binnen hun landbouwsector

Niet alleen milieuregels zijn van belang. Financiële prikkels spelen namelijk ook een grote rol. Overheden beïnvloeden landbouwgedrag niet alleen met wetten, maar ook via subsidies. Die subsidies kunnen duurzame praktijken stimuleren, zoals het beschermen van ecosystemen of het terugdringen van uitstoot. Maar ze kunnen ook onbedoeld vervuilende landbouw ondersteunen, bijvoorbeeld door intensieve veehouderij of het overmatig gebruik van kunstmest aantrekkelijk te maken.


**Twee vormen van landbouwsubsidies:**

| Subsidie | Volledige naam | Inhoud | Duurzaamheidswaarde |
|----------|----------------|--------|----------------------|
| **_GSSE_** | _General Services Support Estimate_ | Overheidsuitgaven voor publieke goederen en langetermijnvoorzieningen in de landbouw (onderzoek, kennisoverdracht, infrastructuur, milieuherstel). Boeren ontvangen deze steun niet direct. |  Wordt algemeen beschouwd als duurzame, niet-productiegebonden steun. |
| **_PSE_**  | _Producer Support Estimate_ | Directe betalingen aan boeren, vaak gekoppeld aan hoeveel ze produceren of exporteren. Dit kan leiden tot intensivering en overproductie. |  Wordt vaak gezien als minder duurzaam of zelfs schadelijk voor het milieu. |

De _TSE_ (_Total Support Estimate_) omvat álle subsidies (dus _TSE_ = _GSSE_ + _PSE_ + andere steunvormen van de originele OECD dataset).

*Bron: [_OECD Glossary on Agricultural Support Indicators_](https://www.oecd.org/agriculture/topics/agricultural-policy-monitoring-and-evaluation/)*

In [ ]:
import pandas as pd
import plotly.graph_objects as go


df = pd.read_csv("../data1/CO2.csv")


landnaam_vertaling = {
    'Australia': 'Australië',
    'China': 'China',
    'Brazil': 'Brazilië',
    'South Africa': 'Zuid-Afrika',
    'United States of America': 'Verenigde Staten',
    'Republic of Korea': 'Zuid-Korea'
}


df['Land_NL'] = df['Mapped_Country'].map(landnaam_vertaling)


df = df.dropna(subset=['GSSE_Subsidy', 'TSE_Subsidy', 'CO2_Emission'])
df = df[df['TSE_Subsidy'] != 0]
df['Duurzaamheidsratio'] = df['GSSE_Subsidy'] / df['TSE_Subsidy']


gewilde_landen = list(landnaam_vertaling.keys())
df = df[df['Mapped_Country'].isin(gewilde_landen)]


df = df.sort_values(by=['Land_NL', 'Year'])
landen = df['Land_NL'].unique()


fig = go.Figure()
dropdown_buttons = []

for i, land in enumerate(landen):
    data = df[df['Land_NL'] == land]
    visible = [False] * (2 * len(landen))

    
    fig.add_trace(go.Bar(
        x=data['Year'],
        y=data['Duurzaamheidsratio'],
        name=f"Duurzame subsidies (GSSE/TSE) – {land}",
        marker_color='#C7EEF9',
        yaxis='y1',
        visible=(i == 0)
    ))
    visible[i * 2] = True

    
    fig.add_trace(go.Scatter(
        x=data['Year'],
        y=data['CO2_Emission'],
        name=f"CO₂-uitstoot – {land}",
        mode='lines+markers',
        marker_color='#EF8C60',
        yaxis='y2',
        visible=(i == 0)
    ))
    visible[i * 2 + 1] = True

    
    dropdown_buttons.append(dict(
        label=land,
        method='update',
        args=[{'visible': visible},
              {'title': f"Duurzame subsidies en CO₂-uitstoot – {land}"}]
    ))


fig.update_layout(
    title="Duurzame subsidies en CO₂-uitstoot – " + landen[0],
    xaxis=dict(title="Jaar"),
    yaxis=dict(title="GSSE / TSE (duurzaamheidsratio)", range=[0, 1]),
    yaxis2=dict(title="CO₂-uitstoot (ton CO₂eq)", overlaying="y", side="right"),
    updatemenus=[dict(
        buttons=dropdown_buttons,
        direction='down',
        x=1.03,
        xanchor="left",
        y=1.15,
        yanchor="top",
        showactive=True
    )],
    legend=dict(x=0.01, y=1),
    barmode='group',
    height=600,
    template='plotly_white'
)

fig.update_layout(
    height=750,
    margin=dict(b=150) 
)


fig.add_annotation(
    text=(
        " Met het dropdownmenu rechtsboven kun je wisselen tussen landen. <br>" 
        " Beweeg met je muis over de grafiek om de exacte waarden per jaar te bekijken. <br>"
        " De grafiek helpt inzicht te krijgen in hoe subsidiebeleid mogelijk samenhangt met de uitstoot uit de landbouwsector."
    ),
    xref="paper", yref="paper",
    x=0, y=-0.25,
    showarrow=False,
    align="left",
    font=dict(size=12),
    borderpad=4
)


fig.show()

FileNotFoundError: [Errno 2] No such file or directory: '../data1/CO2.csv'


<span style="font-size: 60%;">

Deze grafiek vergelijkt per land twee belangrijke grootheden over de tijd:<br><br>

**1. De verhouding GSSE / TSE (Duurzaamheidratio)**<br><br>

De GSSE/TSE-ratio geeft aan welk deel van de totale landbouwsubsidies (TSE) bestaat uit algemene diensten aan de landbouwsector (GSSE), zoals investeringen in duurzaamheid, infrastructuur of milieubescherming.<br><br>

Een hogere GSSE/TSE-ratio betekent dat een groter deel van het landbouwbeleid gericht is op collectieve, vaak duurzamere maatregelen. Landen met een hoge ratio investeren dus relatief meer in bijvoorbeeld ecologische innovatie, bodemkwaliteit of klimaatvriendelijke systemen.<br><br>

**2. Totale CO₂-uitstoot vanuit de landbouwsector**<br><br>

De totale landbouwuitstoot omvat de broeikasgassen die vrijkomen binnen de landbouwsector. Dit wordt uitgedrukt in CO₂-equivalenten (CO₂-eq) om verschillende broeikasgassen, zoals methaan en lachgas, vergelijkbaar te maken op basis van hun effect op het klimaat (Fifth Assessment Report, 2014).<br><br>

Door deze twee lijnen te combineren, wordt zichtbaar of een land dat meer investeert in duurzame subsidies, ook relatief schoner produceert.

</span>



##### Waarom deze landen?
De zes gevisualiseerde landen Australië, Brazilië, China, Zuid-Afrika, Verenigde Staten en Zuid-Korea zijn geselecteerd vanwege hun variatie in duurzaamheidsratio’s. Zo konden zowel landen met een relatief hoge als lage inzet op duurzame landbouwsubsidies worden geanalyseerd, wat waardevol is om verschillen in CO₂-uitstoot beter te begrijpen.

##### Analyse
Uit de visualisaties van zes landen blijkt een opvallende samenhang tussen de mate van duurzame landbouwsubsidies en CO₂-uitstoot. Hoewel we geen causale conclusies kunnen trekken, zijn er wel patronen zichtbaar die het argument ondersteunen dat landen die duurzaamheid belonen via subsidies, stoten relatief minder CO₂ uit binnen hun landbouwsector.


- **Australië** laat een stijgende duurzaamheidsratio zien vanaf de jaren 2000, samen met een licht dalende of stabiele CO₂-uitstoot in recente jaren. Dit wijst op een mogelijke effectiviteit van duurzaam beleid.
  
- **Brazilië** laat zien dat tussen 1995 en 2005 een daling van duurzame subsidies samen gaat met een stijging in CO₂-uitstoot. Wat wel tegenstrijdig lijkt te zijn is dat in 2010 de uitstoot sterk daalt ondanks een lage duurzaamheidsratio.

- **China** De grafiek laat zien dat China tussen 1995 en 2005 een relatief hoge duurzaamheidsratio had. In die periode bleef de CO₂-uitstoot nog relatief laag en groeide slechts langzaam. Maar na 2005 daalt de duurzaamheidsratio flink, terwijl de CO₂-uitstoot juist versneld begint te stijgen. Dat suggereert dat het afbouwen van duurzame subsidies mogelijk heeft bijgedragen aan een snellere stijging van de uitstoot in de landbouwsector

- **Zuid-Afrika** De grafiek van Zuid-Afrika toont een grillig patroon met meerdere pieken en dalen in zowel de duurzaamheidsratio als de CO₂-uitstoot. Opvallend is dat deze schommelingen vaak ongeveer gelijktijdig optreden, wat kan wijzen op een samenhang.

- **Verenigde Staten** hebben langdurig een lage duurzaamheidsratio, namelijk rond 0.1 – 0.2, en de CO₂-uitstoot blijft ook relatief hoog, met recente pieken. De verhouding tussen GSSE en TSE is hier opvallend laag vergeleken met andere landen.

- **Zuid-Korea** In Zuid-Korea blijft de duurzaamheidsratio structureel laag, terwijl de CO₂-uitstoot van de landbouwsector sterk blijft stijgen. Dit ondersteunt het argument dat een gebrek aan gerichte duurzame subsidies samenhangt met een gestage toename van emissies.


De gevisualiseerde grafieken suggereren dat een hogere duurzaamheidratio kan leiden naar relatief lagere CO₂-uitstoot in de landbouwsector.  
De duurzaamheid van subsidies lijkt dus een relevante factor in het klimaatbeleid. De interactieve grafiek maakt dit visueel duidelijk, vooral door het contrast tussen landen met lage en hoge duurzame subsidieratio’s.
